## Chapter 23: Containerization with Docker

Modern applications rarely run in isolation. They depend on databases, message brokers, caching services, and other infrastructure. Ensuring that your application runs consistently across development, testing, and production environments can be challenging. **Containerization** solves this by packaging your application and its dependencies into a lightweight, portable unit called a **container**. **Docker** is the most popular container platform. In this chapter, you’ll learn how to containerize your ASP.NET Core applications using Docker, create efficient `Dockerfile`s, manage multi‑container applications with `docker-compose`, and follow best practices for security and performance. By the end, you’ll be able to build, run, and share containerized .NET applications with confidence.

### 23.1 What Are Containers?

Containers are a form of operating system virtualization. Unlike virtual machines (VMs), which include a full guest OS, containers share the host OS kernel but isolate the application processes. This makes them lightweight, fast to start, and resource‑efficient.

**Key benefits:**

- **Consistency**: The same container image runs identically on any machine with Docker.
- **Isolation**: Containers are isolated from each other and the host, improving security.
- **Portability**: You can run containers on your laptop, on‑prem servers, or any cloud provider.
- **Microservices friendly**: Containers are the perfect unit for deploying microservices.

### 23.2 Docker Basics

Before diving into ASP.NET Core, let’s review essential Docker concepts:

- **Image**: A read‑only template with instructions for creating a container. Images are built from a `Dockerfile`.
- **Container**: A runnable instance of an image. You can start, stop, move, or delete a container.
- **Dockerfile**: A text file with commands to assemble an image.
- **Registry**: A repository for storing and distributing images (e.g., Docker Hub, Azure Container Registry).
- **Docker Compose**: A tool for defining and running multi‑container applications using a YAML file.

### 23.3 Creating a Dockerfile for an ASP.NET Core Application

The official .NET Docker images are provided by Microsoft and are optimized for different scenarios: SDK images for building, and runtime images for running. We’ll use a **multi‑stage build** to keep the final image small.

#### Step 1: Prepare Your Application

Assume you have a standard ASP.NET Core Web API or MVC project. Ensure it builds and runs locally.

#### Step 2: Create a Dockerfile

In the root of your project (where the `.csproj` file is), create a file named `Dockerfile` (no extension) with the following content:

```dockerfile
# Stage 1: Build
FROM mcr.microsoft.com/dotnet/sdk:8.0 AS build
WORKDIR /src

# Copy csproj and restore dependencies (caching layer)
COPY ["MyApp.csproj", "."]
RUN dotnet restore "MyApp.csproj"

# Copy everything else and build
COPY . .
RUN dotnet publish "MyApp.csproj" -c Release -o /app/publish

# Stage 2: Runtime
FROM mcr.microsoft.com/dotnet/aspnet:8.0 AS runtime
WORKDIR /app
EXPOSE 80
EXPOSE 443

# Copy published files from build stage
COPY --from=build /app/publish .

# Set entry point
ENTRYPOINT ["dotnet", "MyApp.dll"]
```

**Explanation:**

- **FROM sdk:8.0 AS build**: Uses the full .NET SDK image to build the application. We name this stage `build`.
- **WORKDIR /src**: Sets the working directory inside the container.
- **COPY ["MyApp.csproj", "."]**: Copies only the project file first to leverage Docker layer caching. If the project file hasn’t changed, Docker will reuse the cached layer for `dotnet restore`.
- **RUN dotnet restore**: Restores NuGet packages.
- **COPY . .**: Copies the rest of the source code.
- **RUN dotnet publish**: Publishes the application in Release mode to `/app/publish`.
- **FROM aspnet:8.0 AS runtime**: Uses the smaller runtime image (no SDK) for the final stage.
- **EXPOSE 80/443**: Documents that the container listens on these ports.
- **COPY --from=build ...**: Copies the published output from the build stage.
- **ENTRYPOINT**: Specifies the command to run when the container starts.

#### Step 3: Build the Docker Image

Open a terminal in the project directory and run:

```bash
docker build -t myapp:latest .
```

- `-t myapp:latest` tags the image with a name and version.
- The `.` indicates the build context (current directory).

#### Step 4: Run the Container

```bash
docker run -d -p 8080:80 --name myapp-container myapp:latest
```

- `-d` runs the container in detached mode (background).
- `-p 8080:80` maps host port 8080 to container port 80.
- `--name` gives the container a name.
- Now open `http://localhost:8080` in your browser.

#### Step 5: Stop and Remove

```bash
docker stop myapp-container
docker rm myapp-container
```

### 23.4 Optimizing Dockerfiles for .NET

- **Use multi‑stage builds** as shown above to keep the final image small (only runtime, not SDK).
- **Copy project files first, then restore** to cache dependencies. This speeds up subsequent builds.
- **Choose the right base image**: `aspnet:8.0` is for runtime; `sdk:8.0` is for building. There are also Alpine‑based variants for even smaller images (e.g., `aspnet:8.0-alpine`), but ensure compatibility.
- **Set `ASPNETCORE_ENVIRONMENT`** via environment variable in the container if needed (e.g., `-e ASPNETCORE_ENVIRONMENT=Production`).

### 23.5 Docker Compose for Multi‑Container Applications

Most real‑world apps need more than just the web server. For example, you might need a SQL Server database and Redis cache. Docker Compose lets you define all services in a single YAML file and start them together.

#### Example `docker-compose.yml`

Place this file in your project root (or a parent directory).

```yaml
version: '3.8'

services:
  db:
    image: mcr.microsoft.com/mssql/server:2022-latest
    environment:
      SA_PASSWORD: "YourStrong!Password"
      ACCEPT_EULA: "Y"
    ports:
      - "1433:1433"
    volumes:
      - sql_data:/var/opt/mssql

  redis:
    image: redis:alpine
    ports:
      - "6379:6379"

  web:
    build:
      context: .
      dockerfile: Dockerfile
    ports:
      - "8080:80"
    environment:
      - ASPNETCORE_ENVIRONMENT=Development
      - ConnectionStrings__DefaultConnection=Server=db;Database=MyAppDb;User Id=sa;Password=YourStrong!Password;TrustServerCertificate=true
      - Redis__ConnectionString=redis:6379
    depends_on:
      - db
      - redis

volumes:
  sql_data:
```

**Explanation:**

- **db**: SQL Server container with environment variables for password and EULA. A named volume `sql_data` persists database files.
- **redis**: Redis container for caching.
- **web**: Builds from the current directory using our `Dockerfile`. It depends on `db` and `redis`, so Compose starts those first. Connection strings reference the service names (`db`, `redis`) which Docker Compose resolves to the container IPs.

#### Running with Docker Compose

```bash
docker-compose up -d
```

This builds the web image (if not already) and starts all containers. Access your app at `http://localhost:8080`.

To stop and remove everything:

```bash
docker-compose down -v   # -v removes volumes
```

### 23.6 Best Practices for Containerizing ASP.NET Core Apps

1. **Run as non‑root user** for security. The official ASP.NET Core images already run as a non‑root user (`app`). You can explicitly switch if needed.
2. **Use `.dockerignore`** to exclude unnecessary files (like `bin`, `obj`, `.git`, `node_modules`) from the build context, speeding up builds and reducing image size.

   Example `.dockerignore`:
   ```
   bin/
   obj/
   .git/
   .vs/
   **/.dockerignore
   Dockerfile
   ```

3. **Keep images small**: Use Alpine‑based images if your app is compatible, and avoid installing unnecessary packages.
4. **Do not store secrets in images**. Use environment variables or Docker secrets (in swarm mode) for sensitive data.
5. **Tag images meaningfully** (e.g., `myapp:1.0.0`, `myapp:latest`) and use version tags in production.
6. **Scan images for vulnerabilities** using tools like `docker scan` or Trivy.
7. **Set resource limits** when running containers (`--memory`, `--cpus`) to prevent a single container from consuming all host resources.

### 23.7 Publishing to a Container Registry

After building your image, you can push it to a registry (Docker Hub, Azure Container Registry, etc.) for deployment.

1. Tag the image with the registry URL:

   ```bash
   docker tag myapp:latest myregistry.azurecr.io/myapp:latest
   ```

2. Log in to the registry:

   ```bash
   docker login myregistry.azurecr.io
   ```

3. Push:

   ```bash
   docker push myregistry.azurecr.io/myapp:latest
   ```

Now your image is available to be pulled on any server.

### 23.8 Running Containers in Production

In production, you’ll likely use an orchestrator like **Kubernetes** or a container platform like **Azure Container Instances** or **AWS ECS**. However, you can also run containers directly on a VM with Docker Engine, managed by a process supervisor or a simple restart policy (`--restart unless-stopped`).

Example run command with restart policy:

```bash
docker run -d --restart unless-stopped -p 80:80 myapp:latest
```

### 23.9 Debugging and Logging

- View logs: `docker logs myapp-container`
- Execute a command inside a running container: `docker exec -it myapp-container bash`
- For ASP.NET Core, logs written to console are captured by Docker and can be viewed with `docker logs`.

### 23.10 Summary

In this chapter, you’ve learned how to containerize your ASP.NET Core applications:

- **Docker** provides lightweight, portable environments.
- **Multi‑stage Dockerfiles** produce small, efficient images.
- **Docker Compose** simplifies running multi‑service applications.
- **Best practices** ensure security, performance, and maintainability.

Containerization is a fundamental skill for modern cloud‑native development. With these techniques, you can ensure your application runs consistently from development to production.

**Exercise:**

1. Create a Dockerfile for your existing ASP.NET Core project (or a new one) using a multi‑stage build.
2. Build the image and run the container, mapping a host port to the container.
3. Create a `docker-compose.yml` that includes your app, a SQL Server container, and a Redis container. Configure your app to use these services.
4. Push your image to a container registry (e.g., Docker Hub) and pull/run it on another machine.

In the next chapter, **"Continuous Integration and Deployment (CI/CD),"** you’ll learn how to automate the build, test, and deployment of your containerized applications using Azure DevOps or GitHub Actions. We’ll cover pipelines, build agents, and deploying to cloud platforms like Azure App Service or Kubernetes.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='22. grpc_vs_signalr_vs_web_apis.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='24. continuous_integration_and_deployment.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
